In [1]:
import pandas as pd
import numpy as np
import time
import sklearn
from sklearn.linear_model import LinearRegression
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import pairwise_distances
from scipy import sparse
from sklearn import preprocessing

In [2]:
cd '/Users/Mengying/Desktop/Networks/Project'

/Users/Mengying/Desktop/Networks/Project


In [3]:
# track_id, song_id mapping
unique_tracks = pd.read_csv('Data/MillionSongSubset/AdditionalFiles/subset_unique_tracks.txt',names = ['track_id','song_id','artist','song'],sep='<SEP>',engine='python')
unique_tracks.head() # 10000

,track_id,song_id,artist,song
0,TRAAAAW128F429D538,SOMZWCG12A8C13C480,Casual,I Didn't Mean To
1,TRAAABD128F429CF47,SOCIWDW12A8C13D406,The Box Tops,Soul Deep
2,TRAAADZ128F9348C2E,SOXVLOJ12AB0189215,Sonora Santanera,Amor De Cabaret
3,TRAAAEF128F4273421,SONHOTT12A8C13493C,Adam Ant,Something Girls
4,TRAAAFD128F92F423A,SOFSOCN12A8C143F5D,Gob,Face the Ashes


In [4]:
profile = pd.read_table('Data/train_triplets.txt', names = ['user','song','count'])
profile.head() # 48,373,586

,user,song,count
0,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOAKIMP12A8C130995,1
1,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOAPDEY12A81C210A9,1
2,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBBMDR12A8C13253B,2
3,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBFNSP12AF72A0E22,1
4,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBFOVM12A58A7D494,1


In [5]:
unique_users = profile.groupby(['user'])['count'].sum()
unique_users.head() #1,019,318

user
00000b722001882066dff9d2da8a775658053ea0    13
00001638d6189236866af9bbf309ae6c2347ffdc    56
0000175652312d12576d9e6b84f600caa24c4715    19
00001cf0dce3fb22b0df0f3a1d9cd21e38385372    25
0000267bde1b3a70ea75cf2b2d216cb828e3202b    27
Name: count, dtype: int64

In [6]:
unique_users.sort_values(ascending=False, inplace = True)
unique_users.tail()

user
0155cff3544ca02cc471d0d4ce1148c7f000ae7e    10
c35b8664d4dc68308e91934e1cdc28d34bee86f6    10
467ea4cb5fa300b650fa6b27b80495b4315db3bb    10
46842492af4e0c8aa6c5d6cb3ea6982873dea4e2    10
98b8b7dd46b150d5181f5674c0f8f686066f3b4e    10
Name: count, dtype: int64

In [7]:
## filtering songs #original: 384,546 --> 772,661
filtered_profile = profile.merge(unique_tracks, left_on='song', right_on='song_id', how='inner')

In [8]:
filtered_unique_users = filtered_profile.groupby(['user'])['count'].sum() #418,252

In [59]:
filtered_profile.head()

,user,song_x,count,track_id,song_id,artist,song_y
0,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOWEZSI12A81C21CE6,1,TRAUCNU128F42671EB,SOWEZSI12A81C21CE6,Gipsy Kings,Tu Quieres Volver
1,833c530ecda3d99deb8395f70400aa3999783d91,SOWEZSI12A81C21CE6,2,TRAUCNU128F42671EB,SOWEZSI12A81C21CE6,Gipsy Kings,Tu Quieres Volver
2,d6c5bd2b570b4faf8964d7ed04f3392ff505d2be,SOWEZSI12A81C21CE6,1,TRAUCNU128F42671EB,SOWEZSI12A81C21CE6,Gipsy Kings,Tu Quieres Volver
3,724534729c9f5dc72a009269c2c225883e4775d2,SOWEZSI12A81C21CE6,1,TRAUCNU128F42671EB,SOWEZSI12A81C21CE6,Gipsy Kings,Tu Quieres Volver
4,ee7aa84c164038c963cfd02a7e52a5598aa470c3,SOWEZSI12A81C21CE6,2,TRAUCNU128F42671EB,SOWEZSI12A81C21CE6,Gipsy Kings,Tu Quieres Volver


In [40]:
# Create train/test split 8:2
np.random.seed(6080)
num_users = filtered_unique_users.shape[0] #418252
train_users = np.random.choice(filtered_unique_users.index, int(np.floor(num_users)*0.8),replace=False)
train = filtered_profile[filtered_profile['user'].isin(train_users)] #(618260, 7),334601 users
test = filtered_profile[~filtered_profile['user'].isin(train_users)] #(154401, 7),83651 users

In [67]:
train['count'].sort_values(ascending=False)

218935    541
346536    501
511403    427
211543    380
189130    377
423131    372
511780    344
689836    329
17852     329
693422    322
719121    321
769540    302
211345    282
160071    282
764108    281
376096    273
552499    268
52911     268
638671    257
639828    256
376505    241
564492    234
404380    231
161337    228
373827    225
373680    221
737911    221
732351    220
106991    216
231193    214
         ... 
459550      1
459489      1
459488      1
459483      1
459448      1
459433      1
459434      1
459435      1
459436      1
459437      1
459438      1
459439      1
459441      1
459442      1
459443      1
459447      1
459454      1
459482      1
459455      1
459460      1
459461      1
459468      1
459471      1
459473      1
459474      1
459476      1
459477      1
459479      1
459481      1
0           1
Name: count, Length: 618260, dtype: int64

### Collaborative Filtering -> Finding similar users

#### Only on Filtered data

In [60]:
# Using filtered
filtered_SONG_AVG = dict(train['count'].groupby(train['track_id']).mean()) # {song: song_avg_count}
filtered_USER_AVG = dict(train['count'].groupby(train['user']).mean())

In [47]:
# Encode user and song id to make integer position 
track_encoder = preprocessing.LabelEncoder()
train['TRACK_ID'] = track_encoder.fit_transform(train['track_id'])
user_encoder = preprocessing.LabelEncoder()
train['USER_ID'] = user_encoder.fit_transform(train['user'])

/anaconda3/envs/si618/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/anaconda3/envs/si618/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


In [53]:
test.track_id.unique().shape[0] # train: 3607, test: 3015, 2947 common track_id

3015

In [69]:
# Check for common track id and cold start
test_track_set = pd.DataFrame(test.track_id.unique())
train_track_set = pd.DataFrame(train.track_id.unique())
test_track_set.merge(train_track_set, how='inner').shape 

,0
0,TRAUCNU128F42671EB
1,TRBFXMJ12903CB50F6
2,TRBBMHE128EF341D09
3,TRAUURC128E078EC6E
4,TRAQVTO128F14696A6


In [55]:
# Create matrix
listening_counts = np.array(train['count'].values)
user_pos = np.array(train['USER_ID'].values)
song_pos = np.array(train['TRACK_ID'].values)
mtx = sparse.coo_matrix((listening_counts, (user_pos,song_pos)), shape=(334601, 3607))
user_song_matrix = mtx.tocsr() #(334601, 3607)

In [ ]:
def predict_CF(data):
    for index, row in data.iterrows():
        try:
            u_ind = filtered_USER_AVG[row['user']]
        except:
            continue
        try:
            m_ind = filtered_SONG_AVG[row['track_id']]
        except:
            continue